In [ ]:
!git clone https://github.com/openai/gpt-2.git

In [ ]:
import os
os.chdir('gpt-2')

In [ ]:
!pip3 install tensorflow-gpu==1.12.0
!pip3 install tensorflow==1.14.0

In [ ]:
!pip3 install -r requirements.txt

In [ ]:
!python3 download_model.py 345M

In [ ]:

!export PYTHONIOENCODING=UTF-8

In [ ]:
os.chdir('src')

In [ ]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder

In [ ]:
def interact_model(raw_text,
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    #print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                return text    
                    


In [ ]:
text=interact_model("I went to a lounge to celebrate my birthday and",
    '345M',
    None,
    1,
    1,
    300,
    1,
    0,
    '/content/gpt-2/models'
)

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import Flask, request
from datetime import datetime
from flask import Flask, request, render_template
app = Flask(__name__,template_folder='/content/gpt-2/src')
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route('/')
def my_form():
    return render_template('form.html')

@app.route('/', methods=['POST'])
def my_form_post():
    input_text = request.form['text']
    length = request.form['length']
    output_text=interact_model(input_text,'345M',None,1,1,int(length),1,0,'/content/gpt-2/models')
    return output_text


    
 
app.run()